Import da biblioteca struct, padrão do python

In [24]:
import struct

Função para carregar o arquivo .BMP e ler as suas informações

In [25]:
def carregar_foto(caminho_arquivo):
    try:
        with open(caminho_arquivo, 'rb') as f:
            file_type_bytes = f.read(2)
            if file_type_bytes != b'BM':
                raise ValueError("Não é um arquivo BMP válido (assinatura 'BM' ausente).")
            
            rest_of_file_header_bytes = f.read(12) 
            bfSize, bfReserved1, bfReserved2, bfOffBits = struct.unpack('<IHH I', rest_of_file_header_bytes)

            file_header_bytes = file_type_bytes + rest_of_file_header_bytes

            info_header_bytes = f.read(40)
            (
                biSize, biWidth, biHeight, biPlanes, biBitCount,
                biCompression, biSizeImage, biXPelsPerMeter,
                biYPelsPerMeter, biClrUsed, biClrImportant
            ) = struct.unpack('<IIIHHIIIIII', info_header_bytes)

            if biBitCount != 24:
                raise ValueError(f"Apenas BMPs de 24 bits são suportados. Encontrado: {biBitCount} bpp.")
            
            f.seek(bfOffBits)

            pixel_data = []
            bytes_per_pixel = 3
            row_padding = (4 - (biWidth * bytes_per_pixel) % 4) % 4

            for y in range(biHeight):
                row_pixels = []
                for x in range(biWidth):
                    b, g, r = struct.unpack('<BBB', f.read(3))
                    row_pixels.append((r, g, b))
                
                f.read(row_padding)
                pixel_data.append(row_pixels)
            
            pixel_data.reverse()

            return pixel_data, biWidth, biHeight, file_header_bytes, info_header_bytes

    except FileNotFoundError:
        return None, None, None, None, None
    except ValueError as e:
        return None, None, None, None, None
    except struct.error as e:
        return None, None, None, None, None
    

Função para salvar as informações e o arquivo .BMP

In [26]:
def salvar_bmp(caminho_arquivo, pixel_data, width, height, original_file_header, original_info_header):
    try:
        with open(caminho_arquivo, 'wb') as f:
            pixels_to_write = list(pixel_data)
            pixels_to_write.reverse() 

            bytes_per_pixel = 3
            row_padding = (4 - (width * bytes_per_pixel) % 4) % 4
            image_data_size = (width * bytes_per_pixel + row_padding) * height
            
            _bfSize, _bfReserved1, _bfReserved2, _bfOffBits = struct.unpack('<IHH I', original_file_header[2:14])
            
            new_file_size = _bfOffBits + image_data_size
            
            new_file_header = bytearray(original_file_header) 
            
            new_file_header[2:6] = struct.pack('<I', new_file_size)
            
            new_file_header[10:14] = struct.pack('<I', _bfOffBits)

            f.write(new_file_header)
            
            f.write(original_info_header) 

            current_pos_after_headers = len(new_file_header) + len(original_info_header)
            if current_pos_after_headers < _bfOffBits:
                f.write(b'\x00' * (_bfOffBits - current_pos_after_headers))

            for y in range(height):
                for x in range(width):
                    r, g, b = pixels_to_write[y][x]
                    f.write(struct.pack('<BBB', b, g, r))
                
                f.write(b'\x00' * row_padding)

            print(f"Imagem salva em '{caminho_arquivo}'.")

    except Exception as e:
        print(f"Erro ao salvar BMP: {e}")



In [27]:
if __name__ == "__main__":
    
    input_file = "lena.bmp" 
    output_grayscale_file = "lena_grayscale.bmp"
    output_bw_file = "lena_bw.bmp"

    image_data, width, height, file_header, info_header = carregar_foto(input_file)

    if image_data is None:
        print("Não foi possível carregar a imagem. Encerrando.")
    else:        
        # 1. Converter para Tons de Cinza
        grayscale_image_data = []
        for y in range(height):
            row_pixels = []
            for x in range(width):
                r, g, b = image_data[y][x]
                gray_value = int(0.299 * r + 0.587 * g + 0.114 * b)
                if gray_value < 0: gray_value = 0
                elif gray_value > 255: gray_value = 255
                row_pixels.append((gray_value, gray_value, gray_value))
            grayscale_image_data.append(row_pixels)

        salvar_bmp(output_grayscale_file, grayscale_image_data, width, height, file_header, info_header)

        # 2. Converter para Preto e Branco
        bw_image_data = []
        threshold = 128 
        for y in range(height):
            row_pixels = []
            for x in range(width):
                gray_value = grayscale_image_data[y][x][0] 
                if gray_value >= threshold:
                    row_pixels.append((255, 255, 255)) 
                else:
                    row_pixels.append((0, 0, 0))     
            bw_image_data.append(row_pixels)

        salvar_bmp(output_bw_file, bw_image_data, width, height, file_header, info_header)

Imagem salva em 'lena_grayscale.bmp'.
Imagem salva em 'lena_bw.bmp'.
